# Dependent Variable: Chart Survival Top 50

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import os
os.chdir('/Users/sunnihermann/Desktop/')
print(os.getcwd())

/Users/sunnihermann/Desktop


In [2]:
top_50 = pd.read_csv('top_50.csv')

In [58]:
chart_survival = top_50.copy()
chart_survival = chart_survival[['country', 'issue_country', 'year', 'id']]
chart_survival.head(5)

,country,issue_country,year,id
0,CO,US,2021,4MzXwWMhyBbmu6hOcLVD49
1,CO,US,2021,4MzXwWMhyBbmu6hOcLVD49
2,CO,US,2021,4MzXwWMhyBbmu6hOcLVD49
3,CO,US,2021,4MzXwWMhyBbmu6hOcLVD49
4,CO,US,2021,4MzXwWMhyBbmu6hOcLVD49


In [59]:
chart_survival = chart_survival.groupby(['country', 'issue_country', 'year'])['id'].count().reset_index()
chart_survival.rename(columns={'id': 'chart_survival'}, inplace=True)
chart_survival.head()

,country,issue_country,year,chart_survival
0,AE,AE,2019,166
1,AE,AE,2020,61
2,AE,AU,2020,27
3,AE,AU,2021,5
4,AE,CA,2019,1


In [60]:
#deleting no_market_countries from the country pairs, as they are not relevant
no_market_country = set(list(chart_survival.issue_country.unique())) - set(list(chart_survival.country.unique()))
chart_survival = chart_survival.loc[~chart_survival['issue_country'].isin(no_market_country)] 

In [61]:
chart_survival.shape

(3288, 4)

In [62]:
chart_survival.chart_survival.value_counts()

1       173
2        87
3        70
4        63
5        59
       ... 
3064      1
2100      1
1282      1
3530      1
1587      1
Name: chart_survival, Length: 1298, dtype: int64

In [63]:
chart_survival.to_csv('chart_survival_top50.csv')

In [64]:
df_model = pd.read_csv('df_final.csv')
df_model.head(3)

,Unnamed: 0,country,issue_country,year,chart_survival,Streams,country_language,issue_country_language,common_language,dist_social,country_iso3,issue_country_iso3,dist_cultural,dist_geo,dist_eco,home_bias
0,0,AE,AE,2019,354,558523,['ar'],['ar'],1,8.951117e-07,ARE,ARE,0.0,108.789,NaN,1
1,1,AE,AE,2020,399,648159,['ar'],['ar'],1,8.951117e-07,ARE,ARE,0.0,108.789,NaN,1
2,2,AE,AE,2021,81,115387,['ar'],['ar'],1,8.951117e-07,ARE,ARE,0.0,108.789,NaN,1


In [65]:
df_model = df_model[['country','issue_country','year','common_language', 'dist_social',
                    'dist_cultural','dist_geo','dist_eco','home_bias']]

In [66]:
df_model = chart_survival.merge(df_model, how = 'left', on = ['country', 'issue_country', 'year'])

In [67]:
print(df_model.shape)
df_model.head(3)

(3288, 10)


,country,issue_country,year,chart_survival,common_language,dist_social,dist_cultural,dist_geo,dist_eco,home_bias
0,AE,AE,2019,166,1,8.951117e-07,0.000000,108.789,NaN,1
1,AE,AE,2020,61,1,8.951117e-07,0.000000,108.789,NaN,1
2,AE,AU,2020,27,0,8.381527e-05,4.186266,11962.663,NaN,0


In [68]:
cols = ['dist_social', 'dist_cultural', 'dist_geo', 'dist_eco']
df_model[cols] = df_model[cols].replace({0:np.nan})

df_model[['chart_survival', 'dist_social', 'dist_cultural', 'dist_geo', 'dist_eco']] = np.log(
    df_model[['chart_survival', 'dist_social', 'dist_cultural', 'dist_geo', 'dist_eco']])

In [69]:
df_model.head()

,country,issue_country,year,chart_survival,common_language,dist_social,dist_cultural,dist_geo,dist_eco,home_bias
0,AE,AE,2019,5.111988,1,-13.926317,NaN,4.689410,NaN,1
1,AE,AE,2020,4.110874,1,-13.926317,NaN,4.689410,NaN,1
2,AE,AU,2020,3.295837,0,-9.386895,1.431809,9.389546,NaN,0
3,AE,AU,2021,1.609438,0,-9.386895,1.431809,9.389546,3.728417,0
4,AE,CA,2019,0.000000,0,-9.515322,1.246133,9.283880,5.119443,0


In [70]:
df_model.shape

(3288, 10)

------

In [71]:
df_model_no_mutual = df_model.copy()

In [72]:
df_model_no_mutual = df_model_no_mutual.loc[df_model_no_mutual['country'] != df_model_no_mutual['issue_country']]
print(df_model_no_mutual.shape)
df_model_no_mutual.head(3)

(3043, 10)


,country,issue_country,year,chart_survival,common_language,dist_social,dist_cultural,dist_geo,dist_eco,home_bias
2,AE,AU,2020,3.295837,0,-9.386895,1.431809,9.389546,NaN,0
3,AE,AU,2021,1.609438,0,-9.386895,1.431809,9.389546,3.728417,0
4,AE,CA,2019,0.000000,0,-9.515322,1.246133,9.283880,5.119443,0


In [73]:
smallest_sample = df_model_no_mutual.copy()
smallest_sample = smallest_sample.dropna()
#smallest_sample = smallest_sample.loc[smallest_sample.chart_survival > 0]
print(smallest_sample.shape)

(2479, 10)


# Model

In [74]:
import gme as gme

gme_data = gme.EstimationData(data_frame = df_model,
                              imp_var_name = 'country',
                              exp_var_name = 'issue_country',
                              trade_var_name = 'chart_survival',
                              year_var_name = 'year')

print(gme_data)

number of countries: 69 
number of exporters: 58 
number of importers: 69 
number of years: 5 
number of sectors: not_applicable 
dimensions: (3288, 10)



In [75]:
gme_data_no_mutual = gme.EstimationData(data_frame = df_model_no_mutual,
                              imp_var_name = 'country',
                              exp_var_name = 'issue_country',
                              trade_var_name = 'chart_survival',
                              year_var_name = 'year')

print(gme_data_no_mutual)

number of countries: 69 
number of exporters: 48 
number of importers: 69 
number of years: 5 
number of sectors: not_applicable 
dimensions: (3043, 10)



In [76]:
gme_smallest_sample = gme.EstimationData(data_frame = smallest_sample,
                              imp_var_name = 'country',
                              exp_var_name = 'issue_country',
                              trade_var_name = 'chart_survival',
                              year_var_name = 'year')

print(gme_smallest_sample)

number of countries: 63 
number of exporters: 46 
number of importers: 63 
number of years: 5 
number of sectors: not_applicable 
dimensions: (2479, 10)



In [77]:
gme_model2 = gme.EstimationModel(estimation_data = gme_data_no_mutual,
                                lhs_var = 'chart_survival',
                                rhs_var = ['common_language', 'dist_geo'], 
                                fixed_effects = [['country', 'year'], ['issue_country', 'year']],
                                keep_years = [2017, 2018, 2019, 2020, 2021]) 

In [78]:
estimates2 = gme_model2.estimate()
results2 = estimates2['all']
#gme_model2.format_regression_table(format = "txt", variable_list=['common_language','dist_geo'] )

Estimation began at 04:39 PM  on Nov 25, 2022
Omitted Regressors: ['issue_country_year_fe_AE2018', 'issue_country_year_fe_IE2019', 'issue_country_year_fe_IS2021', 'issue_country_year_fe_LV2017', 'issue_country_year_fe_US2018', 'issue_country_year_fe_UY2017', 'issue_country_year_fe_UY2019', 'issue_country_year_fe_ZA2020', 'issue_country_year_fe_ZA2021']
Estimation completed at 04:39 PM  on Nov 25, 2022


In [79]:
gme_model2_r = gme.EstimationModel(estimation_data = gme_smallest_sample,
                                lhs_var = 'chart_survival',
                                rhs_var = ['common_language', 'dist_geo'], 
                                fixed_effects = [['country', 'year'], ['issue_country', 'year']],
                                keep_years = [2017, 2018, 2019, 2020, 2021]) 

estimates2_r = gme_model2_r.estimate()
results2_r = estimates2_r['all']

pseudo_rsquared2 = (1 - results2_r.llf / results2_r.llnull)
pseudo_rsquared2

Estimation began at 04:39 PM  on Nov 25, 2022
Omitted Regressors: ['issue_country_year_fe_AE2018', 'issue_country_year_fe_IE2019', 'issue_country_year_fe_IS2021', 'issue_country_year_fe_LV2017', 'issue_country_year_fe_US2018', 'issue_country_year_fe_US2019', 'issue_country_year_fe_UY2017', 'issue_country_year_fe_ZA2020', 'issue_country_year_fe_ZA2021']
Estimation completed at 04:39 PM  on Nov 25, 2022


0.20903326583636495

In [80]:
gme_model4 = gme.EstimationModel(estimation_data = gme_data_no_mutual,
                                lhs_var = 'chart_survival',
                                rhs_var = ['dist_social', 'common_language', 'dist_geo'], 
                                fixed_effects = [['country', 'year'], ['issue_country', 'year']],
                                keep_years = [2017, 2018, 2019, 2020, 2021]) 

In [81]:
estimates4 = gme_model4.estimate()
results4 = estimates4['all']

Estimation began at 04:39 PM  on Nov 25, 2022
Omitted Regressors: ['issue_country_year_fe_AE2018', 'issue_country_year_fe_IE2019', 'issue_country_year_fe_IS2021', 'issue_country_year_fe_LV2017', 'issue_country_year_fe_US2018', 'issue_country_year_fe_UY2017', 'issue_country_year_fe_UY2019', 'issue_country_year_fe_ZA2020', 'issue_country_year_fe_ZA2021']
Estimation completed at 04:39 PM  on Nov 25, 2022


In [82]:
gme_model4_r = gme.EstimationModel(estimation_data = gme_smallest_sample,
                                lhs_var = 'chart_survival',
                                rhs_var = ['dist_social', 'common_language', 'dist_geo'], 
                                fixed_effects = [['country', 'year'], ['issue_country', 'year']],
                                keep_years = [2017, 2018, 2019, 2020, 2021]) 

estimates4_r = gme_model4_r.estimate()
results4_r = estimates4_r['all']

pseudo_rsquared4 = (1 - results4_r.llf / results4_r.llnull)
pseudo_rsquared4

Estimation began at 04:39 PM  on Nov 25, 2022
Omitted Regressors: ['issue_country_year_fe_AE2018', 'issue_country_year_fe_IE2019', 'issue_country_year_fe_IS2021', 'issue_country_year_fe_LV2017', 'issue_country_year_fe_US2018', 'issue_country_year_fe_US2019', 'issue_country_year_fe_UY2017', 'issue_country_year_fe_ZA2020', 'issue_country_year_fe_ZA2021']
Estimation completed at 04:39 PM  on Nov 25, 2022


0.2115906586879881

In [83]:
gme_model6 = gme.EstimationModel(estimation_data = gme_data_no_mutual,
                                lhs_var = 'chart_survival',
                                rhs_var = ['dist_cultural', 'common_language', 'dist_geo'], 
                                fixed_effects = [['country', 'year'], ['issue_country', 'year']],
                                keep_years = [2017, 2018, 2019, 2020, 2021]) 

In [84]:
estimates6 = gme_model6.estimate()
results6 = estimates6['all']

Estimation began at 04:39 PM  on Nov 25, 2022
Omitted Regressors: ['issue_country_year_fe_AE2018', 'issue_country_year_fe_IE2019', 'issue_country_year_fe_IS2021', 'issue_country_year_fe_LV2017', 'issue_country_year_fe_US2018', 'issue_country_year_fe_US2019', 'issue_country_year_fe_UY2017', 'issue_country_year_fe_ZA2020', 'issue_country_year_fe_ZA2021']
Estimation completed at 04:39 PM  on Nov 25, 2022


In [85]:
gme_model6_r = gme.EstimationModel(estimation_data = gme_smallest_sample,
                                lhs_var = 'chart_survival',
                                rhs_var = ['dist_cultural', 'common_language', 'dist_geo'], 
                                fixed_effects = [['country', 'year'], ['issue_country', 'year']],
                                keep_years = [2017, 2018, 2019, 2020, 2021]) 

estimates6_r = gme_model6_r.estimate()
results6_r = estimates6_r['all']

pseudo_rsquared6 = (1 - results6_r.llf / results6_r.llnull)
pseudo_rsquared6

Estimation began at 04:39 PM  on Nov 25, 2022
Omitted Regressors: ['issue_country_year_fe_AE2018', 'issue_country_year_fe_IE2019', 'issue_country_year_fe_IS2021', 'issue_country_year_fe_LV2017', 'issue_country_year_fe_US2018', 'issue_country_year_fe_US2019', 'issue_country_year_fe_UY2017', 'issue_country_year_fe_ZA2020', 'issue_country_year_fe_ZA2021']
Estimation completed at 04:39 PM  on Nov 25, 2022


0.20969739429105683

In [86]:
gme_model8 = gme.EstimationModel(estimation_data = gme_data_no_mutual,
                                lhs_var = 'chart_survival',
                                rhs_var = ['dist_eco', 'common_language', 'dist_geo'], 
                                fixed_effects = [['country', 'year'], ['issue_country', 'year']],
                                keep_years = [2017, 2018, 2019, 2020, 2021]) 

In [87]:
estimates8 = gme_model8.estimate()
results8 = estimates8['all']

Estimation began at 04:40 PM  on Nov 25, 2022
Omitted Regressors: ['issue_country_year_fe_AE2018', 'issue_country_year_fe_IE2019', 'issue_country_year_fe_IS2021', 'issue_country_year_fe_LV2017', 'issue_country_year_fe_US2018', 'issue_country_year_fe_UY2017', 'issue_country_year_fe_UY2019', 'issue_country_year_fe_ZA2020', 'issue_country_year_fe_ZA2021']
Estimation completed at 04:40 PM  on Nov 25, 2022


In [88]:
gme_model8_r = gme.EstimationModel(estimation_data = gme_smallest_sample,
                                lhs_var = 'chart_survival',
                                rhs_var = ['dist_eco', 'common_language', 'dist_geo'], 
                                fixed_effects = [['country', 'year'], ['issue_country', 'year']],
                                keep_years = [2017, 2018, 2019, 2020, 2021]) 

estimates8_r = gme_model8_r.estimate()
results8_r = estimates8_r['all']

pseudo_rsquared8 = (1 - results8_r.llf / results8_r.llnull)
pseudo_rsquared8

Estimation began at 04:40 PM  on Nov 25, 2022
Omitted Regressors: ['issue_country_year_fe_AE2018', 'issue_country_year_fe_IE2019', 'issue_country_year_fe_IS2021', 'issue_country_year_fe_LV2017', 'issue_country_year_fe_US2018', 'issue_country_year_fe_US2019', 'issue_country_year_fe_UY2017', 'issue_country_year_fe_ZA2020', 'issue_country_year_fe_ZA2021']
Estimation completed at 04:40 PM  on Nov 25, 2022


0.2093400035350993

In [89]:
gme_model10 = gme.EstimationModel(estimation_data = gme_data,
                                lhs_var = 'chart_survival',
                                rhs_var = ['home_bias','dist_geo', 'common_language'], 
                                fixed_effects = [['country', 'year'], ['issue_country', 'year']],
                                keep_years = [2017, 2018, 2019, 2020, 2021]) 

In [90]:
estimates10 = gme_model10.estimate()
results10 = estimates10['all']

Estimation began at 04:40 PM  on Nov 25, 2022
Omitted Regressors: ['issue_country_year_fe_AE2018', 'issue_country_year_fe_UY2017', 'issue_country_year_fe_ZA2018', 'issue_country_year_fe_ZA2019', 'issue_country_year_fe_ZA2020', 'issue_country_year_fe_ZA2021']
Estimation completed at 04:40 PM  on Nov 25, 2022


In [91]:
gme_model10_r = gme.EstimationModel(estimation_data = gme_smallest_sample,
                                lhs_var = 'chart_survival',
                                rhs_var = ['home_bias', 'common_language', 'dist_geo'], 
                                fixed_effects = [['country', 'year'], ['issue_country', 'year']],
                                keep_years = [2017, 2018, 2019, 2020, 2021]) 

estimates10_r = gme_model10_r.estimate()
results10_r = estimates10_r['all']

pseudo_rsquared10 = (1 - results10_r.llf / results10_r.llnull)
pseudo_rsquared10

Estimation began at 04:40 PM  on Nov 25, 2022
Omitted Regressors: ['home_bias', 'issue_country_year_fe_AE2018', 'issue_country_year_fe_IE2019', 'issue_country_year_fe_IS2021', 'issue_country_year_fe_LV2017', 'issue_country_year_fe_US2018', 'issue_country_year_fe_US2019', 'issue_country_year_fe_UY2017', 'issue_country_year_fe_ZA2020', 'issue_country_year_fe_ZA2021']
Estimation completed at 04:40 PM  on Nov 25, 2022


0.20903326583636495

In [92]:
#Cultural Distance
gme_model11 = gme.EstimationModel(estimation_data = gme_data_no_mutual,
                                lhs_var = 'chart_survival',
                                rhs_var = ['common_language', 'dist_geo', 'dist_social', 'dist_cultural', 'dist_eco'], #independent variables
                                fixed_effects = [['country', 'year'], ['issue_country', 'year']], #importer and exporter fixed effects
                                keep_years = [2017, 2018, 2019, 2020, 2021] #all years are included
                                )

In [93]:
estimates11 = gme_model11.estimate()
results11 = estimates11['all']

Estimation began at 04:40 PM  on Nov 25, 2022
Omitted Regressors: ['issue_country_year_fe_AE2018', 'issue_country_year_fe_IE2019', 'issue_country_year_fe_IS2021', 'issue_country_year_fe_LV2017', 'issue_country_year_fe_US2018', 'issue_country_year_fe_US2019', 'issue_country_year_fe_UY2017', 'issue_country_year_fe_ZA2020', 'issue_country_year_fe_ZA2021']
Estimation completed at 04:40 PM  on Nov 25, 2022


In [94]:
gme_model11_r = gme.EstimationModel(estimation_data = gme_smallest_sample,
                                lhs_var = 'chart_survival',
                                rhs_var = ['common_language', 'dist_geo', 'dist_social', 'dist_cultural', 'dist_eco'], 
                                fixed_effects = [['country', 'year'], ['issue_country', 'year']],
                                keep_years = [2017, 2018, 2019, 2020, 2021]) 

estimates11_r = gme_model11_r.estimate()
results11_r = estimates11_r['all']

pseudo_rsquared11 = (1 - results11_r.llf / results11_r.llnull)
pseudo_rsquared11

Estimation began at 04:40 PM  on Nov 25, 2022
Omitted Regressors: ['issue_country_year_fe_AE2018', 'issue_country_year_fe_IE2019', 'issue_country_year_fe_IS2021', 'issue_country_year_fe_LV2017', 'issue_country_year_fe_US2018', 'issue_country_year_fe_US2019', 'issue_country_year_fe_UY2017', 'issue_country_year_fe_ZA2020', 'issue_country_year_fe_ZA2021']
Estimation completed at 04:40 PM  on Nov 25, 2022


0.21237529767609198

In [95]:
gme_model12 = gme.EstimationModel(estimation_data = gme_data,
                                lhs_var = 'chart_survival',
                                rhs_var = ['common_language', 'dist_geo', 'dist_social', 'dist_eco', 'home_bias'], #independent variables
                                fixed_effects = [['country', 'year'], ['issue_country', 'year']], #importer and exporter fixed effects
                                keep_years = [2017, 2018, 2019, 2020, 2021] #all years are included
                                )

In [96]:
estimates12 = gme_model12.estimate()
results12 = estimates12['all']

Estimation began at 04:40 PM  on Nov 25, 2022
Omitted Regressors: ['issue_country_year_fe_AE2018', 'issue_country_year_fe_IE2019', 'issue_country_year_fe_IS2021', 'issue_country_year_fe_LV2017', 'issue_country_year_fe_US2018', 'issue_country_year_fe_UY2017', 'issue_country_year_fe_UY2019', 'issue_country_year_fe_ZA2020', 'issue_country_year_fe_ZA2021']
Estimation completed at 04:40 PM  on Nov 25, 2022


In [97]:
gme_model12_r = gme.EstimationModel(estimation_data = gme_smallest_sample,
                                lhs_var = 'chart_survival',
                                rhs_var = ['common_language', 'dist_geo', 'dist_social', 'dist_eco', 'home_bias'], 
                                fixed_effects = [['country', 'year'], ['issue_country', 'year']],
                                keep_years = [2017, 2018, 2019, 2020, 2021]) 

estimates12_r = gme_model12_r.estimate()
results12_r = estimates12_r['all']

pseudo_rsquared12 = (1 - results12_r.llf / results12_r.llnull)
pseudo_rsquared12

Estimation began at 04:40 PM  on Nov 25, 2022
Omitted Regressors: ['home_bias', 'issue_country_year_fe_AE2018', 'issue_country_year_fe_IE2019', 'issue_country_year_fe_IS2021', 'issue_country_year_fe_LV2017', 'issue_country_year_fe_US2018', 'issue_country_year_fe_US2019', 'issue_country_year_fe_UY2017', 'issue_country_year_fe_ZA2020', 'issue_country_year_fe_ZA2021']
Estimation completed at 04:40 PM  on Nov 25, 2022


0.2115943219602937

In [98]:
from stargazer.stargazer import Stargazer

stargazer_tab = Stargazer([results2, results4, results6, results8, results10, results12, results11])

#stargazer_tab = Stargazer([results1, results2, results3])
stargazer_tab